In [ ]:
%%capture
!pip install speechbrain

In [ ]:
%%capture
!pip install mir_eval

In [ ]:
%%capture
!pip install pyloudnorm

In [ ]:
%%capture
!pip install neptune-client

In [ ]:
%%capture
!pip install pytorch-lightning scikit-plot neptune-client neptune-contrib[viz] --upgrade

# SepFormer

[ATTENTION IS ALL YOU NEED IN SPEECH SEPARATION](https://arxiv.org/pdf/2010.13154.pdf)



1.   [Speech Brain](https://github.com/speechbrain/speechbrain)
2. [Modelo Pretrained](https://huggingface.co/speechbrain/sepformer-wsj02mix)
3. [Separation](https://github.com/speechbrain/speechbrain/tree/develop/recipes/WSJ0Mix/separation)
3. [WSJ0Mix](https://github.com/speechbrain/speechbrain/tree/develop/recipes/WSJ0Mix)
4. [Train](https://github.com/speechbrain/speechbrain/blob/f8abef6815d483fe2208df5f41055d73c7ddc3f7/recipes/WSJ0Mix/separation/train.py)
5. [Config o SepFormer.py](https://github.com/speechbrain/speechbrain/blob/develop/recipes/WSJ0Mix/separation/hparams/sepformer.yaml)
6.https://github.com/speechbrain/speechbrain/blob/develop/speechbrain/pretrained/interfaces.py


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/Shareddrives/TG-Separación-Fuentes/code/train-speech-separation-models/train

/content/drive/Shareddrives/TG-Separación-Fuentes/code/train-speech-separation-models/train


In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display, Audio
from speechbrain.pretrained import SepformerSeparation as separator
import torchaudio
import os
import sys
import torch
import torch.nn.functional as F
import torchaudio
import speechbrain as sb
import speechbrain.nnet.schedulers as schedulers
from speechbrain.utils.distributed import run_on_main
from torch.cuda.amp import autocast
from hyperpyyaml import load_hyperpyyaml
import numpy as np
from tqdm import tqdm
import csv
import logging
from pytorch_lightning.loggers.neptune import NeptuneLogger

# 0. Configurar Paths

In [ ]:
# Save checkpoints
default_root_dir = "/content/drive/Shareddrives/TG-Separación-Fuentes/code/Checkpoints-separation-models/SepFormer/"
save_best_model = "/content/drive/Shareddrives/TG-Separación-Fuentes/code/Checkpoints-separation-models/SepFormer/best_model/"
model_path = default_root_dir+'sepformer-wsj02mix/'
savedir = model_path+"Examples/"

In [ ]:
PATH_DATA_ROOT = "../../Datasets/01-Data_experimental/intermediate"
MIX = PATH_DATA_ROOT+"mix/"
S_1 = PATH_DATA_ROOT+"source_1/"
S_2 = PATH_DATA_ROOT+"source_2/"

# CSV

ROOT_CSV  ="/content/drive/Shareddrives/TG-Separación-Fuentes/code/"
PATH_SEPFORMER_CSV_TRAIN = ROOT_CSV+"SEPFORMER_mixture_train_mix_clean_callfriend_spanish.csv"
PATH_SEPFORMER_CSV_VALID =ROOT_CSV+"SEPFORMER_mixture_val_mix_clean_callfriend_spanish.csv"
PATH_SEPFORMER_CSV_TEST = ROOT_CSV+"SEPFORMER_mixture_test_mix_clean_callfriend_spanish.csv"

PATH_SEPFORMER_CSV_TEST_TEMPORAL = ROOT_CSV+"SEPFORMER_mixture_test_mix_clean_callfriend_spanishtemporal.csv"

PATH_CONFIG = "./resources/sepformer.yaml"


df_train = pd.read_csv(PATH_SEPFORMER_CSV_TRAIN)
df_val = pd.read_csv(PATH_SEPFORMER_CSV_VALID)
df_test = pd.read_csv(PATH_SEPFORMER_CSV_TEST)
#df_test_temporal = pd.read_csv(PATH_SEPFORMER_CSV_TEST_TEMPORAL)

In [ ]:
df_train.shape

(280714, 14)

# 0. Configurar logger Neptune

In [ ]:
start = 0
end = 17000

In [ ]:
argv = ['/content/drive/Shareddrives/TG-Separación-Fuentes/code/train-speech-separation-models/train/resources/sepformer.yaml']
hparams_file, run_opts, overrides = sb.parse_arguments(argv)
with open(hparams_file) as fin:
    hparams = load_hyperpyyaml(fin, overrides)

hparams["dataloader_opts"]["batch_size"]=1
hparams["dataloader_opts"]["num_workers"]=1
hparams["test_data"] = "/content/drive/Shareddrives/TG-Separación-Fuentes/code/SEPFORMER_mixture_test_mix_clean_callfriend_spanish_"+str(start)+"_"+str(end)+".csv"
hparams["dataloader_opts"]

{'batch_size': 1, 'num_workers': 1}

In [ ]:
hparams["test_data"]

'/content/drive/Shareddrives/TG-Separación-Fuentes/code/SEPFORMER_mixture_test_mix_clean_callfriend_spanish_0_17000.csv'

In [ ]:
# Configurarmos el experimento y sus parametros
experiment_name = "SepFormer_77_train_24.2_val_24.2_test_sta_"+str(start)+"_end_"+str(end)
params=hparams
tags = ["test_lote_start_"+str(start)+"_end_"+str(end)+"_SepFormer_pretrained"]

# Definir Logger 
neptune_logger = NeptuneLogger(
    api_key="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI5NjRkMmY2YS04M2EwLTRiMGMtODk1Ny1mMWQxZTA3NGM1NzAifQ==",
    project_name="josearangos/Tg-speech-separation",experiment_name=experiment_name,tags=tags, close_after_fit=False)

NeptuneLogger will work in online mode


# 1.Test modelo pretrained

In [ ]:
model_before = separator.from_hparams(source="speechbrain/sepformer-wsj02mix",run_opts={"device":"cuda"} ,savedir=model_path)

In [ ]:
model_before.device

'cuda'

In [ ]:
PATH_TEST = df_val.loc[1,"mix_wav"]

In [ ]:
batch, fs_file = torchaudio.load(PATH_TEST)
est_sources = model_before.separate_batch(batch)
est_sources = est_sources / est_sources.max(dim=1, keepdim=True)[0]
display(Audio(PATH_TEST))
display(Audio(est_sources[:, :, 0].detach().cpu(),rate=8000))
display(Audio(est_sources[:, :, 1].detach().cpu(),rate=8000))

# 1.1 Calcular metrics con modelo pretrained

In [ ]:
train_data, valid_data, test_data = dataio_prep(hparams)

In [ ]:
test_loader = sb.dataio.dataloader.make_dataloader(
            test_data, **hparams["dataloader_opts"]
)

In [ ]:
len(test_loader)

17000

In [ ]:
type(hparams)

dict

In [ ]:
predictions,targets,row,mixture = calculate_metrics(start,end,model_before,hparams,test_loader)

  0%|          | 0/17000 [00:00<?, ?it/s]

https://app.neptune.ai/josearangos/Tg-speech-separation/e/TGSPEEC-58


100%|██████████| 17000/17000 [7:09:14<00:00,  1.51s/it]


# 2. Configurar modelo

In [ ]:
def dataio_prep(hparams):
    """Creates data processing pipeline"""

    # 1. Define datasets
    train_data = sb.dataio.dataset.DynamicItemDataset.from_csv(
        csv_path=hparams["train_data"],
        replacements={"data_root": hparams["data_folder"]},
    )

    valid_data = sb.dataio.dataset.DynamicItemDataset.from_csv(
        csv_path=hparams["valid_data"],
        replacements={"data_root": hparams["data_folder"]},
    )

    test_data = sb.dataio.dataset.DynamicItemDataset.from_csv(
        csv_path=hparams["test_data"],
        replacements={"data_root": hparams["data_folder"]},
    )

    datasets = [train_data, valid_data, test_data]

    # 2. Provide audio pipelines

    @sb.utils.data_pipeline.takes("mix_wav")
    @sb.utils.data_pipeline.provides("mix_sig")
    def audio_pipeline_mix(mix_wav):
        mix_sig = sb.dataio.dataio.read_audio(mix_wav)
        return mix_sig

    @sb.utils.data_pipeline.takes("s1_wav")
    @sb.utils.data_pipeline.provides("s1_sig")
    def audio_pipeline_s1(s1_wav):
        s1_sig = sb.dataio.dataio.read_audio(s1_wav)
        return s1_sig

    @sb.utils.data_pipeline.takes("s2_wav")
    @sb.utils.data_pipeline.provides("s2_sig")
    def audio_pipeline_s2(s2_wav):
        s2_sig = sb.dataio.dataio.read_audio(s2_wav)
        return s2_sig

    if hparams["num_spks"] == 3:

        @sb.utils.data_pipeline.takes("s3_wav")
        @sb.utils.data_pipeline.provides("s3_sig")
        def audio_pipeline_s3(s3_wav):
            s3_sig = sb.dataio.dataio.read_audio(s3_wav)
            return s3_sig

    if "wham" in hparams["data_folder"]:

        @sb.utils.data_pipeline.takes("noise_wav")
        @sb.utils.data_pipeline.provides("noise_sig")
        def audio_pipeline_noise(noise_wav):
            noise_sig = sb.dataio.dataio.read_audio(noise_wav)
            return noise_sig

    sb.dataio.dataset.add_dynamic_item(datasets, audio_pipeline_mix)
    sb.dataio.dataset.add_dynamic_item(datasets, audio_pipeline_s1)
    sb.dataio.dataset.add_dynamic_item(datasets, audio_pipeline_s2)
    if hparams["num_spks"] == 3:
        sb.dataio.dataset.add_dynamic_item(datasets, audio_pipeline_s3)
        sb.dataio.dataset.set_output_keys(
            datasets, ["id", "mix_sig", "s1_sig", "s2_sig", "s3_sig"]
        )
    else:
        if "wham" in hparams["data_folder"]:
            print("Using the WHAM! dataset")
            sb.dataio.dataset.add_dynamic_item(datasets, audio_pipeline_noise)
            sb.dataio.dataset.set_output_keys(
                datasets, ["id", "mix_sig", "s1_sig", "s2_sig", "noise_sig"]
            )
        else:
            sb.dataio.dataset.set_output_keys(
                datasets, ["id", "mix_sig", "s1_sig", "s2_sig"]
            )

    return train_data, valid_data, test_data

In [ ]:
# Define training procedure
class Separation(sb.Brain):
    def compute_forward(self, mix, targets, stage, noise=None):
        """Forward computations from the mixture to the separated signals."""

        # Unpack lists and put tensors in the right device
        mix, mix_lens = mix

        print(mix.shape,mix_lens.shape)

        mix, mix_lens = mix.to(self.device), mix_lens.to(self.device)

        # Convert targets to tensor
        targets = torch.cat(
            [targets[i][0].unsqueeze(-1) for i in range(self.hparams.num_spks)],
            dim=-1,
        ).to(self.device)

        # Add speech distortions
        if stage == sb.Stage.TRAIN:
            with torch.no_grad():
                if self.hparams.use_speedperturb or self.hparams.use_rand_shift:
                    mix, targets = self.add_speed_perturb(targets, mix_lens)

                    if "whamr" in self.hparams.data_folder:
                        targets = self.hparams.reverb(
                            targets[0].t(), torch.ones(targets.size(-1))
                        )
                        targets = targets.t().unsqueeze(0)
                        mix = targets.sum(-1)

                    if "wham" in self.hparams.data_folder:
                        noise = noise.to(self.device)
                        len_noise = noise.shape[1]
                        len_mix = mix.shape[1]
                        min_len = min(len_noise, len_mix)

                        # add the noise
                        mix = mix[:, :min_len] + noise[:, :min_len]

                        # fix the length of targets also
                        targets = targets[:, :min_len, :]

                if self.hparams.use_wavedrop:
                    mix = self.hparams.wavedrop(mix, mix_lens)

                if self.hparams.limit_training_signal_len:
                    mix, targets = self.cut_signals(mix, targets)

        # Separation
        mix_w = self.hparams.Encoder(mix)
        est_mask = self.hparams.MaskNet(mix_w)
        mix_w = torch.stack([mix_w] * self.hparams.num_spks)
        sep_h = mix_w * est_mask

        # Decoding
        est_source = torch.cat(
            [
                self.hparams.Decoder(sep_h[i]).unsqueeze(-1)
                for i in range(self.hparams.num_spks)
            ],
            dim=-1,
        )

        # T changed after conv1d in encoder, fix it here
        T_origin = mix.size(1)
        T_est = est_source.size(1)
        if T_origin > T_est:
            est_source = F.pad(est_source, (0, 0, 0, T_origin - T_est))
        else:
            est_source = est_source[:, :T_origin, :]

        return est_source, targets

    def compute_objectives(self, predictions, targets):
        """Computes the sinr loss"""
        return self.hparams.loss(targets, predictions)

    def fit_batch(self, batch):
        """Trains one batch"""
        # Unpacking batch list
        mixture = batch.mix_sig
        targets = [batch.s1_sig, batch.s2_sig]
        if "wham" in self.hparams.data_folder:
            noise = batch.noise_sig[0]
        else:
            noise = None

        if self.hparams.num_spks == 3:
            targets.append(batch.s3_sig)

        if self.hparams.auto_mix_prec:
            with autocast():
                predictions, targets = self.compute_forward(
                    mixture, targets, sb.Stage.TRAIN, noise
                )
                loss = self.compute_objectives(predictions, targets)

                # hard threshold the easy dataitems
                if self.hparams.threshold_byloss:
                    th = self.hparams.threshold
                    loss_to_keep = loss[loss > th]
                    if loss_to_keep.nelement() > 0:
                        loss = loss_to_keep.mean()
                else:
                    loss = loss.mean()

            if (
                loss < self.hparams.loss_upper_lim and loss.nelement() > 0
            ):  # the fix for computational problems
                self.scaler.scale(loss).backward()
                if self.hparams.clip_grad_norm >= 0:
                    self.scaler.unscale_(self.optimizer)
                    torch.nn.utils.clip_grad_norm_(
                        self.modules.parameters(), self.hparams.clip_grad_norm,
                    )
                self.scaler.step(self.optimizer)
                self.scaler.update()
            else:
                self.nonfinite_count += 1
                logger.info(
                    "infinite loss or empty loss! it happened {} times so far - skipping this batch".format(
                        self.nonfinite_count
                    )
                )
                loss.data = torch.tensor(0).to(self.device)
        else:
            predictions, targets = self.compute_forward(
                mixture, targets, sb.Stage.TRAIN, noise
            )
            loss = self.compute_objectives(predictions, targets)

            if self.hparams.threshold_byloss:
                th = self.hparams.threshold
                loss_to_keep = loss[loss > th]
                if loss_to_keep.nelement() > 0:
                    loss = loss_to_keep.mean()
            else:
                loss = loss.mean()

            if (
                loss < self.hparams.loss_upper_lim and loss.nelement() > 0
            ):  # the fix for computational problems
                loss.backward()
                if self.hparams.clip_grad_norm >= 0:
                    torch.nn.utils.clip_grad_norm_(
                        self.modules.parameters(), self.hparams.clip_grad_norm
                    )
                self.optimizer.step()
            else:
                self.nonfinite_count += 1
                logger.info(
                    "infinite loss or empty loss! it happened {} times so far - skipping this batch".format(
                        self.nonfinite_count
                    )
                )
                loss.data = torch.tensor(0).to(self.device)
        self.optimizer.zero_grad()

        return loss.detach().cpu()

    def evaluate_batch(self, batch, stage):
        """Computations needed for validation/test batches"""
        snt_id = batch.id
        mixture = batch.mix_sig
        targets = [batch.s1_sig, batch.s2_sig]
        if self.hparams.num_spks == 3:
            targets.append(batch.s3_sig)

        with torch.no_grad():
            predictions, targets = self.compute_forward(mixture, targets, stage)
            loss = self.compute_objectives(predictions, targets)

        # Manage audio file saving
        if stage == sb.Stage.TEST and self.hparams.save_audio:
            if hasattr(self.hparams, "n_audio_to_save"):
                if self.hparams.n_audio_to_save > 0:
                    self.save_audio(snt_id[0], mixture, targets, predictions)
                    self.hparams.n_audio_to_save += -1
            else:
                self.save_audio(snt_id[0], mixture, targets, predictions)

        return loss.detach()

    def on_stage_end(self, stage, stage_loss, epoch):
        """Gets called at the end of a epoch."""
        # Compute/store important stats
        stage_stats = {"si-snr": stage_loss}
        if stage == sb.Stage.TRAIN:
            self.train_stats = stage_stats

        # Perform end-of-iteration things, like annealing, logging, etc.
        if stage == sb.Stage.VALID:

            # Learning rate annealing
            if isinstance(
                self.hparams.lr_scheduler, schedulers.ReduceLROnPlateau
            ):
                current_lr, next_lr = self.hparams.lr_scheduler(
                    [self.optimizer], epoch, stage_loss
                )
                schedulers.update_learning_rate(self.optimizer, next_lr)
            else:
                # if we do not use the reducelronplateau, we do not change the lr
                current_lr = self.hparams.optimizer.optim.param_groups[0]["lr"]

            self.hparams.train_logger.log_stats(
                stats_meta={"epoch": epoch, "lr": current_lr},
                train_stats=self.train_stats,
                valid_stats=stage_stats,
            )
            self.checkpointer.save_and_keep_only(
                meta={"si-snr": stage_stats["si-snr"]}, min_keys=["si-snr"],
            )
        elif stage == sb.Stage.TEST:
            self.hparams.train_logger.log_stats(
                stats_meta={"Epoch loaded": self.hparams.epoch_counter.current},
                test_stats=stage_stats,
            )

    def add_speed_perturb(self, targets, targ_lens):
        """Adds speed perturbation and random_shift to the input signals"""

        min_len = -1
        recombine = False

        if self.hparams.use_speedperturb:
            # Performing speed change (independently on each source)
            new_targets = []
            recombine = True

            for i in range(targets.shape[-1]):
                new_target = self.hparams.speedperturb(
                    targets[:, :, i], targ_lens
                )
                new_targets.append(new_target)
                if i == 0:
                    min_len = new_target.shape[-1]
                else:
                    if new_target.shape[-1] < min_len:
                        min_len = new_target.shape[-1]

            if self.hparams.use_rand_shift:
                # Performing random_shift (independently on each source)
                recombine = True
                for i in range(targets.shape[-1]):
                    rand_shift = torch.randint(
                        self.hparams.min_shift, self.hparams.max_shift, (1,)
                    )
                    new_targets[i] = new_targets[i].to(self.device)
                    new_targets[i] = torch.roll(
                        new_targets[i], shifts=(rand_shift[0],), dims=1
                    )

            # Re-combination
            if recombine:
                if self.hparams.use_speedperturb:
                    targets = torch.zeros(
                        targets.shape[0],
                        min_len,
                        targets.shape[-1],
                        device=targets.device,
                        dtype=torch.float,
                    )
                for i, new_target in enumerate(new_targets):
                    targets[:, :, i] = new_targets[i][:, 0:min_len]

        mix = targets.sum(-1)
        return mix, targets

    def cut_signals(self, mixture, targets):
        """This function selects a random segment of a given length within the mixture.
        The corresponding targets are selected accordingly"""
        randstart = torch.randint(
            0,
            1 + max(0, mixture.shape[1] - self.hparams.training_signal_len),
            (1,),
        ).item()
        targets = targets[
            :, randstart : randstart + self.hparams.training_signal_len, :
        ]
        mixture = mixture[
            :, randstart : randstart + self.hparams.training_signal_len
        ]
        return mixture, targets

    def reset_layer_recursively(self, layer):
        """Reinitializes the parameters of the neural networks"""
        if hasattr(layer, "reset_parameters"):
            layer.reset_parameters()
        for child_layer in layer.modules():
            if layer != child_layer:
                self.reset_layer_recursively(child_layer)

    def save_results(self, test_data):
        """This script computes the SDR and SI-SNR metrics and saves
        them into a csv file"""

        # This package is required for SDR computation
        from mir_eval.separation import bss_eval_sources

        # Create folders where to store audio
        save_file = os.path.join(self.hparams.output_folder, "test_results.csv")

        # Variable init
        all_sdrs = []
        all_sdrs_i = []
        all_sisnrs = []
        all_sisnrs_i = []
        csv_columns = ["snt_id", "sdr", "sdr_i", "si-snr", "si-snr_i"]

        test_loader = sb.dataio.dataloader.make_dataloader(
            test_data, **self.hparams.dataloader_opts
        )

        with open(save_file, "w") as results_csv:
            writer = csv.DictWriter(results_csv, fieldnames=csv_columns)
            writer.writeheader()

            # Loop over all test sentence
            with tqdm(test_loader, dynamic_ncols=True) as t:
                for i, batch in enumerate(t):

                    # Apply Separation
                    mixture, mix_len = batch.mix_sig
                    snt_id = batch.id
                    targets = [batch.s1_sig, batch.s2_sig]
                    if self.hparams.num_spks == 3:
                        targets.append(batch.s3_sig)

                    with torch.no_grad():
                        predictions, targets = self.compute_forward(
                            batch.mix_sig, targets, sb.Stage.TEST
                        )

                    # Compute SI-SNR
                    sisnr = self.compute_objectives(predictions, targets)

                    # Compute SI-SNR improvement
                    mixture_signal = torch.stack(
                        [mixture] * self.hparams.num_spks, dim=-1
                    )
                    mixture_signal = mixture_signal.to(targets.device)
                    sisnr_baseline = self.compute_objectives(
                        mixture_signal, targets
                    )
                    sisnr_i = sisnr - sisnr_baseline

                    # Compute SDR
                    sdr, _, _, _ = bss_eval_sources(
                        targets[0].t().cpu().numpy(),
                        predictions[0].t().detach().cpu().numpy(),
                    )

                    sdr_baseline, _, _, _ = bss_eval_sources(
                        targets[0].t().cpu().numpy(),
                        mixture_signal[0].t().detach().cpu().numpy(),
                    )

                    sdr_i = sdr.mean() - sdr_baseline.mean()

                    # Saving on a csv file
                    row = {
                        "snt_id": snt_id[0],
                        "sdr": sdr.mean(),
                        "sdr_i": sdr_i,
                        "si-snr": -sisnr.item(),
                        "si-snr_i": -sisnr_i.item(),
                    }
                    writer.writerow(row)

                    # Metric Accumulation
                    all_sdrs.append(sdr.mean())
                    all_sdrs_i.append(sdr_i.mean())
                    all_sisnrs.append(-sisnr.item())
                    all_sisnrs_i.append(-sisnr_i.item())

                row = {
                    "snt_id": "avg",
                    "sdr": np.array(all_sdrs).mean(),
                    "sdr_i": np.array(all_sdrs_i).mean(),
                    "si-snr": np.array(all_sisnrs).mean(),
                    "si-snr_i": np.array(all_sisnrs_i).mean(),
                }
                writer.writerow(row)

        logger.info("Mean SISNR is {}".format(np.array(all_sisnrs).mean()))
        logger.info("Mean SISNRi is {}".format(np.array(all_sisnrs_i).mean()))
        logger.info("Mean SDR is {}".format(np.array(all_sdrs).mean()))
        logger.info("Mean SDRi is {}".format(np.array(all_sdrs_i).mean()))


    def save_audio(self, snt_id, mixture, targets, predictions):
        "saves the test audio (mixture, targets, and estimated sources) on disk"

        # Create outout folder
        save_path = os.path.join(self.hparams.save_folder, "audio_results")
        if not os.path.exists(save_path):
            os.mkdir(save_path)

        for ns in range(self.hparams.num_spks):

            # Estimated source
            signal = predictions[0, :, ns]
            signal = signal / signal.abs().max()
            save_file = os.path.join(
                save_path, "item{}_source{}hat.wav".format(snt_id, ns + 1)
            )
            torchaudio.save(
                save_file, signal.unsqueeze(0).cpu(), self.hparams.sample_rate
            )

            # Original source
            signal = targets[0, :, ns]
            signal = signal / signal.abs().max()
            save_file = os.path.join(
                save_path, "item{}_source{}.wav".format(snt_id, ns + 1)
            )
            torchaudio.save(
                save_file, signal.unsqueeze(0).cpu(), self.hparams.sample_rate
            )

        # Mixture
        signal = mixture[0][0, :]
        signal = signal / signal.abs().max()
        save_file = os.path.join(save_path, "item{}_mix.wav".format(snt_id))
        torchaudio.save(
            save_file, signal.unsqueeze(0).cpu(), self.hparams.sample_rate
        )   


# 4. DataLoaders

In [ ]:
train_data, valid_data, test_data = dataio_prep(hparams)

In [ ]:
len(train_data),len(test_data)

(280714, 5)

# 5. Modelo

In [ ]:
# Brain class initialization
separator = Separation(
    modules=hparams["modules"],
    opt_class=hparams["optimizer"],
    hparams=hparams,
    run_opts=run_opts,
    checkpointer=hparams["checkpointer"],
)

In [ ]:
# re-initialize the parameters
for module in separator.modules.values():
    separator.reset_layer_recursively(module)

In [ ]:
!nvidia-smi

Sat Jun  5 04:56:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    33W / 250W |  16159MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 6. Train

In [ ]:
if not hparams["test_only"]:
  # Training
  separator.fit(
      separator.hparams.epoch_counter,
      train_data,
      valid_data,
      train_loader_kwargs=hparams["dataloader_opts"],
      valid_loader_kwargs=hparams["dataloader_opts"],
  )

# 7.Eval

In [ ]:
def compute_objectives(predictions, targets):
    """Computes the sinr loss"""
    return hparams["loss"](targets, predictions)

In [ ]:
def calculate_metrics(start,end,model_before,hparams,test_loader):
  
  ii = start
  #print("II ==>",ii)

  from mir_eval.separation import bss_eval_sources
  # Create folders where to store audio
  save_file = os.path.join(hparams["output_folder"], "test_results.csv")
  # Variable init
  all_sdrs = []
  all_sdrs_i = []
  all_sisnrs = []
  all_sisnrs_i = []
  csv_columns = ["snt_id", "sdr", "sdr_i", "si-snr", "si-snr_i"]

  with tqdm(test_loader, dynamic_ncols=True) as t:
    for i, batch in enumerate(t):
      

      #print("ITERACCCIÓN ",type(ii),ii)
      neptune_logger.experiment.log_metric("Iteración:", ii)


      # Apply Separation
      mixture, mix_len = batch.mix_sig
      snt_id = batch.id
      targets = [batch.s2_sig,batch.s1_sig]
      with torch.no_grad():
        predictions = model_before.separate_batch(mixture)
        predictions = predictions / predictions.max(dim=1, keepdim=True)[0]

      
      targets = torch.cat(
            [targets[i][0].unsqueeze(-1) for i in range(hparams["num_spks"])],
            dim=-1,
        ).to(model_before.device)
      
      # Compute SI-SNR
      sisnr = compute_objectives(predictions, targets)

      

      # Compute SI-SNR improvement
      mixture_signal = torch.stack(
                [mixture] * hparams["num_spks"], dim=-1
            )
      
      mixture_signal = mixture_signal.to(targets.device)
      sisnr_baseline =compute_objectives(
          mixture_signal, targets
      )

      sisnr_i = sisnr - sisnr_baseline



      # Compute SDR
      sdr, _, _, _ = bss_eval_sources(
                        targets[0].t().cpu().numpy(),
                        predictions[0].t().detach().cpu().numpy(),
                    )
      
      sdr_baseline, _, _, _ = bss_eval_sources(
                        targets[0].t().cpu().numpy(),
                        mixture_signal[0].t().detach().cpu().numpy(),
                    )

      sdr_i = sdr.mean() - sdr_baseline.mean()

      row = {
           "snt_id": snt_id[0],
            "sdr": sdr.mean(),
            "sdr_i": sdr_i,
            "si-snr": -sisnr.item(),
            "si-snr_i": -sisnr_i.item(),
        }
      
      neptune_logger.experiment.log_metric('sdr', round(sdr.mean(),2))
      neptune_logger.experiment.log_metric('sdr_i', round(sdr_i.mean(),2))
      neptune_logger.experiment.log_metric('sisnr', round(-sisnr.item(),2))
      neptune_logger.experiment.log_metric('sisnr_i', round(-sisnr_i.item(),2))
      # row tiene las metrícas del audio
      
      
      # Metric Accumulation
      all_sdrs.append(sdr.mean())
      all_sdrs_i.append(sdr_i.mean())
      all_sisnrs.append(-sisnr.item())
      all_sisnrs_i.append(-sisnr_i.item())

      ii = ii + 1



  row = {
          "snt_id": ["avg"],
          "sdr": [np.array(all_sdrs).mean()],
          "sdr_i": [np.array(all_sdrs_i).mean()],
          "si-snr": [np.array(all_sisnrs).mean()],
          "si-snr_i": [np.array(all_sisnrs_i).mean()],
      }

  
  #row tiene el promedio de las metrícas.
  summary_metrics = pd.DataFrame.from_dict(row)
  path_metrics = hparams["output_folder"]+"summary_metrics_"+str(start)+"_"+str(end)+".csv"

  summary_metrics.to_csv(path_metrics)

  neptune_logger.experiment.log_artifact(path_metrics)
  neptune_logger.experiment.stop()
  return predictions,targets,row,mixture